In [43]:
from __future__ import division      #除数可以显示为float

from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd

import time                 #使用time stamp
import datetime             

import matplotlib.pyplot as plt

import math

# 最基本的KNN算法
from sklearn.neighbors import KNeighborsClassifier

# 导入样本拆分模块
from sklearn.model_selection import train_test_split

# 导入KNN半径算法
from sklearn.neighbors import RadiusNeighborsClassifier


定义学习曲线绘画函数


In [44]:
from sklearn.learning_curve import learning_curve #c查看是否过拟合

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=1, train_sizes=np.linspace(.05, 1., 20), verbose=0, plot=True):
    """
    画出data在某模型上的learning curve.
    参数解释
    ----------
    estimator : 你用的分类器。
    title : 表格的标题。
    X : 输入的feature，numpy类型
    y : 输入的target vector
    ylim : tuple格式的(ymin, ymax), 设定图像中纵坐标的最低点和最高点
    cv : 做cross-validation的时候，数据分成的份数，其中一份作为cv集，其余n-1份作为training(默认为3份)
    n_jobs : 并行的的任务数(默认1)
    """
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, verbose=verbose)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    if plot:
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
        plt.xlabel(u"train_sample")
        plt.ylabel(u"score")
        plt.gca().invert_yaxis()
        plt.grid()

        plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, 
                         alpha=0.1, color="b")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, 
                         alpha=0.1, color="r")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label=u"train_score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label=u"cross_validation_score")

        plt.legend(loc="best")

        plt.draw()
        plt.show()
        plt.gca().invert_yaxis()
        plt.savefig("learn_curve.jpg") 

读取文件


In [45]:
# 读取归一化后的数据
Data_Csv = pd.DataFrame()
Data_Csv = pd.read_csv("Data_Normalize.csv")


# 生成测试数据
X =  np.array(Data_Csv.loc[:,['MACD_macd', 'EMA_gap', 'KDJ_J', 'RSI', 'MOM', 'MOM_gap', 'OBV', \
        'Long_Short_Rate_OBV', 'KDJ_J_pre', 'MOM_pre', 'MOM_gap_pre', \
       'OBV_pre', 'Long_Short_Rate_OBV_pre', 'MACD_macd_pre','EMA_gap_pre']])

# 生成测试结果
y = np.array(Data_Csv.loc[:,"win_rate"])

k = 2

生成测试数据

In [46]:
# 分别生成训练数据、测试数据
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)


开始进行模型训练


In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score


models = []
models.append(("KNN", KNeighborsClassifier(n_neighbors=k)))
models.append(("KNN with weights", KNeighborsClassifier(
    n_neighbors=k, weights="distance")))
models.append(("Radius Neighbors", RadiusNeighborsClassifier(
    n_neighbors=k, radius=25)))

results = []
for name, model in models:
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test);
    
    results.append((name, precision_score(y_test,y_pred ,average='binary')))
    
for i in range(len(results)):
    print("name: {}; score: {}".format(results[i][0],results[i][1]))

name: KNN; score: 0.260869565217
name: KNN with weights; score: 0.280821917808
name: Radius Neighbors; score: 0.0


交叉训练，更加精准预测


In [48]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# 增加多项式预处理
def polynomial_model(degree=1, **kwarg):
    polynomial_features = PolynomialFeatures(degree=degree,
                                             include_bias=False)
    knn_regression = KNeighborsClassifier(n_neighbors=5)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("knn_regression", knn_regression)])
    return pipeline

model = polynomial_model(degree=4, penalty='l2')

start = time.clock()
model.fit(x_train, y_train)


y_pred = model.predict(x_test);
    
print precision_score(y_test,y_pred ,average='binary')

# train_score = model.score(x_train, y_train)
# cv_score = model.score(x_test, y_test)
# print('elaspe: {0:.6f}; train_score: {1:0.6f}; cv_score: {2:.6f}'.format(
#     time.clock()-start, train_score, cv_score))



0.333333333333
